# Use Snowpark_OpenAI Notebook template

In [1]:
!!pip install "snowflake-connector-python[pandas]"

['Collecting snowflake-connector-python[pandas]',
 '  Using cached https://files.pythonhosted.org/packages/f8/c7/03d2ca5e460acb3a31d70ee17584fbc6fe828a52c9f83bbb20a392b8988b/snowflake_connector_python-3.7.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl',
 'Collecting requests<3.0.0',
 '  Using cached https://files.pythonhosted.org/packages/70/8e/0e2d847013cb52cd35b38c009bb167a1a26b2ce6cd6965bf26b47bc0bf44/requests-2.31.0-py3-none-any.whl',
 'Collecting tomlkit',
 '  Using cached https://files.pythonhosted.org/packages/07/fa/c96545d741f2fd47f565e4e06bfef0962add790cb9c2289d900102b55eca/tomlkit-0.12.4-py3-none-any.whl',
 'Collecting cryptography<43.0.0,>=3.1.0',
 '  Using cached https://files.pythonhosted.org/packages/d4/fa/057f9d7a5364c86ccb6a4bd4e5c58920dcb66532be0cc21da3f9c7617ec3/cryptography-42.0.5-cp39-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl',
 'Collecting urllib3<2.0.0,>=1.21.1; python_version < "3.10"',
 '  Using cached https://files.pythonhosted.org/package

In [2]:
!pip install -q matplotlib seaborn xgboost

ERROR: snowflake-ml-python 1.0.10 has requirement packaging<24,>=20.9, but you'll have packaging 24.0 which is incompatible.
ERROR: snowflake-ml-python 1.0.10 has requirement pandas<2,>=1.0.0, but you'll have pandas 2.2.1 which is incompatible.
ERROR: snowflake-ml-python 1.0.10 has requirement xgboost<2,>=1.7.3, but you'll have xgboost 2.0.3 which is incompatible.
ERROR: jupyterlab 3.2.4 has requirement jupyter-server~=1.4, but you'll have jupyter-server 2.0.0a1 which is incompatible.
ERROR: jupyterlab-server 2.25.3 has requirement jsonschema>=4.18.0, but you'll have jsonschema 3.2.0 which is incompatible.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [30]:
!pip3 install -q lightgbm

ERROR: Exception:
Traceback (most recent call last):
  File "/opt/conda/lib/python3.9/site-packages/pip/_internal/cli/base_command.py", line 153, in _main
    status = self.run(options, args)
  File "/opt/conda/lib/python3.9/site-packages/pip/_internal/commands/install.py", line 382, in run
    resolver.resolve(requirement_set)
  File "/opt/conda/lib/python3.9/site-packages/pip/_internal/legacy_resolve.py", line 201, in resolve
    self._resolve_one(requirement_set, req)
  File "/opt/conda/lib/python3.9/site-packages/pip/_internal/legacy_resolve.py", line 365, in _resolve_one
    abstract_dist = self._get_abstract_dist_for(req_to_install)
  File "/opt/conda/lib/python3.9/site-packages/pip/_internal/legacy_resolve.py", line 312, in _get_abstract_dist_for
    abstract_dist = self.preparer.prepare_linked_requirement(
  File "/opt/conda/lib/python3.9/site-packages/pip/_internal/operations/prepare.py", line 223, in prepare_linked_requirement
    abstract_dist = _get_prepared_distribution(
 

In [1]:
import pandas as pd # package for high-performance, easy-to-use data structures and data analysis
import numpy as np # fundamental package for scientific computing with Python
import matplotlib
import matplotlib.pyplot as plt # for plotting

import seaborn as sns # for making plots with seaborn
color = sns.color_palette()

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

import datetime

# Preprocessing, modelling and evaluating
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, KFold
from xgboost import XGBClassifier
import xgboost as xgb

# Read Customer and Payment data from Snowflake respective tables

In [2]:
#Import all snowflake connection details from template variables.

db_user = os.getenv('sf_user')
db_password =  'Password@2023' #os.getenv('sf_password')
db_account = os.getenv('sf_account')
db_database =  os.getenv('sf_db')
db_role = os.getenv('sf_role')
db_warehouse = 'FOSFOR_FDC'
db_schema = 'PUBLIC'

In [3]:
from snowflake.snowpark.session import Session
connection_params = {
    'user': db_user,
    'password': db_password,
    'account': db_account,
    'warehouse': db_warehouse,
    'database': db_database,
    'schema': db_schema,
    'role': db_role
}
session = Session.builder.configs(connection_params).create()

In [4]:
session.sql('use warehouse FOSFOR_FDC;').collect()
session.sql('use database FDC_DATA_MANISH;').collect()
session.sql('use schema FDC_DATA_MANISH.PUBLIC;').collect()

cc_customer = session.table('FDC_DATA_MANISH.PUBLIC.CC_CUSTOMER_DATA').to_pandas()
cc_payment = session.table('FDC_DATA_MANISH.PUBLIC.CC_PAYMENT_DATA').to_pandas()

# Reverse Data Engineering to get Raw data

In [5]:
cc_customer['FEA_2'] = cc_customer['FEA_2'].replace([0],np.nan)

In [6]:
cc_payment['PROD_LIMIT'] = cc_payment['PROD_LIMIT'].replace([0],np.nan)
cc_payment['HIGHEST_BALANCE'] = cc_payment['HIGHEST_BALANCE'].replace([0],np.nan)

cc_payment['UPDATE_DATE'] = cc_payment['UPDATE_DATE'].replace(['31/12/9999'],np.nan)
cc_payment['REPORT_DATE'] = cc_payment['REPORT_DATE'].replace(['31/12/9999'],np.nan)

# Check Missing Value/data

In [7]:
# checking missing data
def get_missing_data(df):
    total = df.isnull().sum().sort_values(ascending = False)
    percent = (df.isnull().sum()/df.isnull().count()*100).sort_values(ascending = False)
    percent = round(percent,2)
    missing_df  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    print( missing_df.head(20))
    return None

In [8]:
get_missing_data(cc_customer)

        Total  Percent
FEA_2     149    13.24
LABEL       0     0.00
ID          0     0.00
FEA_1       0     0.00
FEA_3       0     0.00
FEA_4       0     0.00
FEA_5       0     0.00
FEA_6       0     0.00
FEA_7       0     0.00
FEA_8       0     0.00
FEA_9       0     0.00
FEA_10      0     0.00
FEA_11      0     0.00


In [9]:
get_missing_data(cc_payment)

                 Total  Percent
PROD_LIMIT        6118    74.16
REPORT_DATE       1114    13.50
HIGHEST_BALANCE    409     4.96
UPDATE_DATE         26     0.32
OVD_T3               0     0.00
OVD_T2               0     0.00
OVD_T1               0     0.00
ID                   0     0.00
PROD_CODE            0     0.00
PAY_NORMAL           0     0.00
OVD_SUM              0     0.00
NEW_BALANCE          0     0.00


In [10]:
def data_profile(df):
    stats = []
    for col in df.columns:
        stats.append((col, df[col].nunique(), df[col].isnull().sum() * 100 / df.shape[0], df[col].value_counts(normalize=True, dropna=False).values[0] * 100, df[col].dtype))

    stats_df = pd.DataFrame(stats, columns=['Feature', 'Unique_values', 'Percentage of missing values', 'Percentage of values in the biggest category', 'type'])
    stats_df.sort_values('Percentage of missing values', ascending=True)
    return stats_df

In [11]:
data_profile(cc_customer)

,Feature,Unique_values,Percentage of missing values,Percentage of values in the biggest category,type
0,LABEL,2,0.000000,80.000000,float64
1,ID,1125,0.000000,0.088889,float64
2,FEA_1,6,0.000000,42.311111,float64
3,FEA_2,158,13.244444,13.244444,float64
4,FEA_3,3,0.000000,60.800000,float64
5,FEA_4,229,0.000000,3.022222,float64
6,FEA_5,2,0.000000,92.888889,float64
7,FEA_6,10,0.000000,41.333333,float64
8,FEA_7,10,0.000000,61.244444,float64
9,FEA_8,52,0.000000,8.711111,float64


In [12]:
data_profile(cc_payment)

,Feature,Unique_values,Percentage of missing values,Percentage of values in the biggest category,type
0,ID,1125,0.000000,0.666667,int32
1,OVD_T1,21,0.000000,90.606061,int8
2,OVD_T2,16,0.000000,95.587879,int8
3,OVD_T3,33,0.000000,96.763636,int8
4,OVD_SUM,393,0.000000,88.848485,int16
5,PAY_NORMAL,37,0.000000,11.030303,int8
6,PROD_CODE,21,0.000000,54.824242,int8
7,PROD_LIMIT,321,74.157576,74.157576,float64
8,UPDATE_DATE,3041,0.315152,0.315152,object
9,NEW_BALANCE,3939,0.000000,46.836364,float64


In [13]:
cc_customer['FEA_2'] = cc_customer['FEA_2'].replace([np.nan],cc_customer['FEA_2'].mean())

In [14]:
cc_payment['HIGHEST_BALANCE'] = cc_payment['HIGHEST_BALANCE'].replace([np.nan],0)

In [15]:
cc_customer.columns

Index(['LABEL', 'ID', 'FEA_1', 'FEA_2', 'FEA_3', 'FEA_4', 'FEA_5', 'FEA_6',
       'FEA_7', 'FEA_8', 'FEA_9', 'FEA_10', 'FEA_11'],
      dtype='object')

In [16]:
Total_features=['FEA_1', 'FEA_2', 'FEA_3', 'FEA_4', 'FEA_5', 'FEA_6',
       'FEA_7', 'FEA_8', 'FEA_9', 'FEA_10', 'FEA_11']

In [17]:
cat_features = ['FEA_3','FEA_5','FEA_6','FEA_9']
num_features = ['FEA_1','FEA_2','FEA_4','FEA_7','FEA_8','FEA_10','FEA_11']

In [18]:
def standerdisation(df,num_cols):
# copy of datasets
    df_stand = df.copy()

    # apply standardization on numerical features
    for i in num_cols:

        # fit on training data column
        scale = StandardScaler().fit(df_stand[[i]])

        # transform the training data column
        df_stand[i] = scale.transform(df_stand[[i]])
        
    return df_stand

In [19]:
cc_customer_std =  standerdisation(cc_customer,num_features)

In [20]:
feat_cols=[cols for cols  in cc_customer_std.columns if cols not in ['LABEL','ID']]

# Preparing data for model

In [21]:
X_train, X_test, y_train, y_test=train_test_split(cc_customer_std[feat_cols],cc_customer_std[['LABEL']],test_size=0.15)

In [24]:
X_train.head()

,index,FEA_1,FEA_2,FEA_3,FEA_4,FEA_5,FEA_6,FEA_7,FEA_8,FEA_9,FEA_10,FEA_11
0,0,1.097352,0.416866,3.0,0.091809,2.0,11.0,0.056269,0.183361,3.0,-0.087361,0.233161
1,1,1.097352,0.510247,3.0,4.288361,2.0,11.0,1.403133,-1.318688,5.0,-0.686229,2.241331
2,2,-0.349070,-0.143420,1.0,0.238858,2.0,15.0,-1.964028,0.850939,4.0,1.871948,0.555094
3,3,1.097352,-0.765960,3.0,-0.564256,1.0,11.0,1.403133,-0.484216,4.0,-0.686065,0.584161
4,4,1.097352,-0.890468,3.0,-0.654748,2.0,11.0,0.056269,0.934386,4.0,-0.685862,-1.190396


In [25]:
y_train.head()

,index,LABEL
0,0,0.0
1,1,1.0
2,2,0.0
3,3,0.0
4,4,0.0


In [26]:
X_train.reset_index(inplace=True)
X_test.reset_index(inplace=True)
y_train.reset_index(inplace=True)
y_test.reset_index(inplace=True)

X_train.drop(columns=['index'],axis=1,inplace=True)
X_test.drop(columns=['index'],axis=1,inplace=True)
y_train.drop(columns=['index'],axis=1,inplace=True)
y_test.drop(columns=['index'],axis=1,inplace=True)

print('Shape of Train features :-',X_train.shape)
print('Shape of Test features  :-',X_test.shape)

print('Shape of Train Target   :-',y_train.shape)
print('Shape of Test Target    :-',y_test.shape)
y_train = y_train['LABEL'].copy()
y_test = y_test['LABEL'].copy()

Shape of Train features :- (956, 12)
Shape of Test features  :- (169, 12)
Shape of Train Target   :- (956, 2)
Shape of Test Target    :- (169, 2)


# Defining Hyperparameters for Model

In [27]:
params = {'application': 'binary',
          'boosting': 'gbdt',
          'metric': 'auc',
          'max_depth': 16,
          'learning_rate': 0.05,
          'bagging_fraction': 0.9,
          'feature_fraction': 0.9,
          'verbosity': -1,
          'lambda_l1': 0.1,
          'lambda_l2': 0.01,
          'num_leaves': 500,
          'min_child_weight': 3,
          'data_random_seed': 17,
         'nthreads':4}


early_stop = 500
verbose_eval = 30
num_rounds = 600

folds = 5
seed =10
kf = KFold(n_splits = folds, shuffle = True, random_state=seed)
debug =False

# Modeling

In [ ]:
y_preds2 = np.zeros(X_test.shape[0])
feature_importance_df = pd.DataFrame()
i = 0
for tr_idx, val_idx in kf.split(X_train, y_train):

    train_X = X_train[feat_cols].iloc[tr_idx]
    val_X = X_train[feat_cols].iloc[val_idx]
    
    train_y = y_train.iloc[tr_idx]
    val_y = y_train.iloc[val_idx]
    
    lgb_train = lgb.Dataset(train_X, train_y, categorical_feature=cat_features)
    lgb_eval = lgb.Dataset(val_X, val_y, categorical_feature=cat_features)
    
    model = lgb.train(params,
                      train_set=lgb_train,
                      num_boost_round=num_rounds,
                      valid_sets=(lgb_train, lgb_eval),
                      verbose_eval=100)
        
    
    y_preds2+= model.predict(X_test) / folds
    
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["Feature"] = train_X.columns
    fold_importance_df["importance"] = model.feature_importance()
    fold_importance_df["fold"] = i + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    if debug:    
        print("debug:",roc_auc_score(y_test, model.predict(X_test) / folds))  
    i+=1
    del train_X,lgb_train

In [28]:
import LightGBM as lgb

ModuleNotFoundError: No module named 'LightGBM'